In [ ]:
# instal ffmpeg
sudo apt-get install ffmpeg

In [ ]:
!sudo pip install pydub
!sudo pip install google-cloud-speech

In [22]:
import logging
import os
import pickle

from pydub import AudioSegment
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
from google.cloud import storage


In [15]:



def m4a_2_wav(m4a_bucket, wav_bucket):
    """
    Converts all m4a in a bucket to a wav in copy to another bucket
    """
    ori_ext = '.m4a'
    tmp_name = 'file.m4a'
    
    
    client = storage.Client()
    bucket = client.get_bucket(m4a_bucket)

    blobs = bucket.list_blobs()
    for blob in blobs:
        print(blob.name)
        if blob.name.endswith(ori_ext):
            with open(tmp_name, 'wb') as fh:
                blob.download_to_file(fh)
            track = AudioSegment.from_file(tmp_name, 'm4a')
            file_handle = track.export('file.wav', format='wav')
            # copy to bucket
            bucket_out = client.get_bucket(wav_bucket)        
            filename, _ = os.path.splitext(blob.name)
            blob_new_name = filename + '.wav'
            blob = bucket_out.blob(blob_new_name)
            blob.upload_from_filename(filename='file.wav')
        
            

In [17]:
def get_text(bucket, blob, sample_rate_hertz=48000, language_code='es-ES'):
    """
    Get one wav blob and con
    """
    client = speech_v1.SpeechClient()
    # This field is optional for FLAC and WAV audio formats.
    encoding = enums.RecognitionConfig.AudioEncoding.LINEAR16
    metadata = {
        "original_media_type": enums.RecognitionMetadata.OriginalMediaType.AUDIO,
        "original_mime_type": 'audio/m4a',}
    config = {
        "metadata": metadata,
        "audio_channel_count": 2,
        "language_code": language_code,
        "enable_separate_recognition_per_channel": True,
        }
    audio = {"uri": "gs://{}/{}".format(bucket, blob)}
    print("gs://{}/{}".format(bucket, blob))
    operation = client.long_running_recognize(config, audio)
    print(u"Waiting for operation to complete...{}".format(language_code))
    return operation.result()

In [23]:

def transcribe_bucket(bucket_name, language_code='es-ES'):
    """
    Transcribe all wav files from a bucket
    """

    tmp_pickle_name = 'data.pickle'

    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blobs = bucket.list_blobs()
    for blob in blobs:
        # check for pickle version
        filename, _ = os.path.splitext(blob.name)
        pickled_name = filename + '.pickle'
        if blob.name.endswith('.wav') and not bucket.blob(pickled_name).exists():
            # Generate transcription
            transcribed = get_text(bucket.name, blob.name, language_code=language_code)
            # Upload raw output to a pickle
            with open(tmp_pickle_name, 'wb') as fh:
                # Pickle the transcription using the highest protocol available.
                pickle.dump(transcribed, fh)
            blob = bucket.blob(pickled_name)
            blob.upload_from_filename(tmp_pickle_name)
        else:
            print("no transcription will be done for: {}".format(blob.name))
        # Generate clear text
        #break
    logging.warning('DONE')
        

In [8]:
def transcript_object_2_text_file(t_obj, filename):
    """
    
    """
    channel_1 = ''
    channel_2 = ''
    for result in t_obj.results:
        #print(result)
        if result.channel_tag == 1:
            channel_1 += result.alternatives[0].transcript + ' '
        elif result.channel_tag == 2:
            channel_2 += result.alternatives[0].transcript + ' '
    outtext = 'Transcribed Text: \n\nChannel 1: '
    outtext += channel_1 + '\n---\nChannel 2: '
    outtext += channel_2 + '\n'
    with open(filename, 'w') as fh:
        fh.write(outtext)
    return None

In [24]:
# from pickle to text

def batch_pickle_2_text(bucket):
    """
    Convert all pickle files into text
    """
    tmp_txt_name = 'tmp.txt'
    tmp_pickle_name = 'tmp.pickle'

    client = storage.Client()
    bucket = client.get_bucket(bucket)

    blobs = bucket.list_blobs()
    for blob in blobs:
        if blob.name.endswith('.pickle'):
            # download
            with open(tmp_pickle_name, 'wb') as fh:
                blob.download_to_file(fh)
            # 
            with open(tmp_pickle_name, 'rb') as fh:
                #pickle_obj = fh.read()
                obj = pickle.load(fh)
            transcript_object_2_text_file(obj, tmp_txt_name)
            filename, _ = os.path.splitext(blob.name)
            filename += '.txt'
            blob = bucket.blob(filename)
            blob.upload_from_filename(tmp_txt_name)
    logging.warning("DONE")

In [ ]:
# Run conversion

out_bucket = "audioout_es-es"

m4a_2_wav("original_audios", out_bucket)
transcribe_bucket(out_bucket, 'es-ES')
batch_pickle_2_text(out_bucket)